In [ ]:
import numpy as np
import pandas as pd
import random
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.dpi'] = 300

In [ ]:
df_45 = pd.read_csv('stoich45_fingerprints.csv')
df_120 = pd.read_csv('stoich120_fingerprints.csv')
df_sine = pd.read_csv('sine_matrix_fingerprints.csv')
df_qmof_refcodes = pd.read_csv('qmof-refcodes.csv')
df_ofm_fp = pd.read_csv('ofm_fingerprints.csv')

df_bandgaps = pd.read_csv('qmof-bandgaps.csv')
df_bandgaps = df_bandgaps.rename(columns={'refcode': 'MOF'})

#Run this for issues with google collab (i.e MOF not exisiting in the Data frame)

In [ ]:
if 'MOF' in df_120.columns:
    columns = ['MOF'] + [col for col in df_120.columns if col != 'MOF']
    df_120 = df_120[columns]
else:
    raise KeyError("The column 'MOF' does not exist in the DataFrame.")

if 'MOF' in df_45.columns:
    columns = ['MOF'] + [col for col in df_120.columns if col != 'MOF']
    df_120 = df_120[columns]
else:
    raise KeyError("The column 'MOF' does not exist in the DataFrame.")

if 'MOF' in df_bandgaps.columns:
    columns = ['MOF'] + [col for col in df_120.columns if col != 'MOF']
    df_120 = df_120[columns]
else:
    raise KeyError("The column 'MOF' does not exist in the DataFrame.")

In [ ]:
#check difference in between 45 and 120 dataset
# Extract the column names (features)
features_45 = set(df_45.columns)
features_120 = set(df_120.columns)

# Find features in df1 that are not in df2
unique_features_45 = features_45 - features_120

# Print the unique features
print("Features in df1 that are not in df2:")
print(unique_features_45)
print(len(unique_features_45))




```
# This is formatted as code
```

**Exploring the 45 fingerprints**

In [ ]:
#df_45.head(10)
#display all the columns, all of the 45 features
pd.options.display.max_columns = None
df_45.head()

In [ ]:
df_45.info()

In [ ]:
df_45.shape

In [ ]:
#Plotting histograms of each of the 45 fingerprints.

numeric_cols = df_45.select_dtypes(include='number').columns

num_cols = 5
num_rows = (len(numeric_cols) + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 4 * num_rows))

axes = axes.flatten()

for i, (col, ax) in enumerate(zip(numeric_cols, axes)):
    df_45[col].hist(ax=ax, bins=20)
    ax.set_title(col)
    ax.set_xlabel('Values')
    ax.set_ylabel('Frequency')

for j in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
#Plotting Box plots for each 45 features
numerical_columns = df_45.select_dtypes(include=['number']).columns

# Plot box plots for numerical features in a 9 by 5 grid
plt.figure(figsize=(20, 30))  # Adjust figure size for 9 by 5 grid
for i, feature in enumerate(numerical_columns, start=1):
    plt.subplot(9, 5, i)
    plt.boxplot(df_45[feature])
    plt.title(feature)
    plt.grid(True)
    plt.xticks([])  # Remove x-axis labels to save space

plt.tight_layout()
plt.show()


**Exploring the target variable - BG_PBE (Band gaps)**


In [ ]:
df_bandgaps.head()

In [ ]:
df_bandgaps.shape

In [ ]:
df_bandgaps.info()

In [ ]:
# Visualizing the distribution of the band gaps.
df_bandgaps['BG_PBE'].hist(bins=20)
plt.xlabel('Bandgap Values (eV)')
plt.ylabel('Values Count')
plt.title('Distribution of Bandgaps')
plt.show()

In [ ]:
# Box plot of BG_PBE
plt.figure(figsize=(8, 6))
plt.boxplot(df_bandgaps['BG_PBE'])
plt.title('Box Plot for BG_PBE')
plt.ylabel('Feature Values')
plt.grid(True)
plt.show()


In [ ]:
# Correlation coefficients of BG_PBE against 47 numerical features.

df_45 = df_45.set_index(['MOF'])
df_bandgaps = df_bandgaps.set_index('MOF')
df_45_bandgap_joined = df_45.join(df_bandgaps, how='outer', rsuffix='_bandgaps') #joining the 45 fingerprints with the bandgaps data

numeric_cols = df_45_bandgap_joined.select_dtypes(include='number').columns

df_spearman = df_45_bandgap_joined[[col for col in numeric_cols if col.endswith(("_max", "_min")) or col == 'BG_PBE']]
df_pearson = df_45_bandgap_joined[[col for col in numeric_cols if not col.endswith(("_max", "_min"))]]

spearman_matrix = df_spearman.corr(method='spearman')
pearson_matrix = df_pearson.corr(method='pearson')

bandgap_spearman_correlation = spearman_matrix['BG_PBE'].abs().sort_values(ascending=False)
bandgap_pearson_correlation = pearson_matrix['BG_PBE'].abs().sort_values(ascending=False)
print("Spearman:")
print(bandgap_spearman_correlation)
print("Pearson:")
print(bandgap_pearson_correlation)

# Plotting the correlation matrices
plt.figure(figsize=(12, 10))
sns.heatmap(spearman_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Spearman Correlation Matrix for the 45 fingerprints')
plt.show()

plt.figure(figsize=(20, 16))
sns.heatmap(pearson_matrix, annot=True, cmap='coolwarm', fmt='.2f',annot_kws={"size": 8})
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.title('Pearson Correlation Matrix for the 45 fingerprints')
plt.show()

In [ ]:
# These are the missing values in the 45 feature representation.

df_45_bandgap_nan = df_45_bandgap_joined[df_45_bandgap_joined.isna().any(axis=1)]

nan_columns = df_45_bandgap_nan.columns[df_45_bandgap_nan.isna().any()]
df_45_bandgap_nan[nan_columns]

In [ ]:
df_45.tail()

In [ ]:
# Scatter plots with correlation coefficients of BG_PBE against 47 features.
from scipy.stats import pearsonr
from scipy.stats import spearmanr

df_corr_plot = df_45_bandgap_joined.fillna(0) # there are missing values in some of the columns, here we zero them.

numeric_cols = df_corr_plot.select_dtypes(include='number').columns

num_cols = 6
num_rows = (len(numeric_cols) + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(5*num_cols, 4*num_rows))

for i, col in enumerate(numeric_cols):
    x = df_corr_plot['BG_PBE']
    y = df_corr_plot[col]
    if col.endswith(("_max", "_min")):
        corr = ('Spearman', round(spearmanr(x, y)[0], 2))
    else:
        corr = ('Pearson', round(pearsonr(x, y)[0], 2))
    ax = axes[i // num_cols][i % num_cols]
    ax.scatter(x, y, label=f"{corr[0]}={corr[1]}", alpha=0.2)
    ax.set_xlabel('BG_PBE')
    ax.set_ylabel(col)
    ax.legend()

for j in range(len(numeric_cols), num_rows*num_cols):
    fig.delaxes(axes[j // num_cols][j % num_cols])

plt.tight_layout()
plt.show()

**Exploring the 120 feature representation**

In [ ]:
#df_120.head(3)
#display all the columns, all of the 120 features
pd.options.display.max_columns = None
df_120.head()

In [ ]:
df_120.info()

In [ ]:
df_120.shape

In [ ]:
#Plotting histograms of each of the 120 fingerprints.

numeric_cols = df_120.select_dtypes(include='number').columns

num_cols = 5
num_rows = (len(numeric_cols) + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 4 * num_rows))

axes = axes.flatten()

for i, (col, ax) in enumerate(zip(numeric_cols, axes)):
    df_45[col].hist(ax=ax, bins=20)
    ax.set_title(col)
    ax.set_xlabel('Values')
    ax.set_ylabel('Frequency')

for j in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
# Correlation coefficients of BG_PBE against 120 numerical features.

df_120 = df_120.reset_index().set_index(['MOF'])
df_bandgaps = df_bandgaps.reset_index().set_index('MOF')
df_120_bandgap_joined = df_120.join(df_bandgaps, how='outer', rsuffix='_bandgaps') #joining the 120 fingerprints with the bandgaps data

numeric_cols = df_120_bandgap_joined.select_dtypes(include='number').columns

df_spearman = df_120_bandgap_joined[[col for col in numeric_cols if col.endswith(("_max", "_min")) or col == 'BG_PBE']]
df_pearson = df_120_bandgap_joined[[col for col in numeric_cols if not col.endswith(("_max", "_min"))]]

spearman_matrix = df_spearman.corr(method='spearman')
pearson_matrix = df_pearson.corr(method='pearson')

bandgap_spearman_correlation = spearman_matrix['BG_PBE'].abs().sort_values(ascending=False)
bandgap_pearson_correlation = pearson_matrix['BG_PBE'].abs().sort_values(ascending=False)
print("Spearman:")
print(bandgap_spearman_correlation)
print("Pearson:")
print(bandgap_pearson_correlation)

In [ ]:
df_120.index

In [ ]:
df_120.columns

In [ ]:
# These are the missing values in the 120 feature representation.

df_120_bandgap_nan = df_120_bandgap_joined[df_120_bandgap_joined.isna().any(axis=1)]

nan_columns = df_120_bandgap_nan.columns[df_120_bandgap_nan.isna().any()]
df_120_bandgap_nan[nan_columns]

In [ ]:
# Scatter plots with correlation coefficients of BG_PBE against 120 features.
from scipy.stats import pearsonr
from scipy.stats import spearmanr

df_corr_plot = df_120_bandgap_joined.fillna(0) # there are missing values in some of the columns, here we zero them.

numeric_cols = df_corr_plot.select_dtypes(include='number').columns

num_cols = 6
num_rows = (len(numeric_cols) + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(5*num_cols, 4*num_rows))

for i, col in enumerate(numeric_cols):
    x = df_corr_plot['BG_PBE']
    y = df_corr_plot[col]
    if col.endswith(("_max", "_min")):
        corr = ('Spearman', round(spearmanr(x, y)[0], 2))
    else:
        corr = ('Pearson', round(pearsonr(x, y)[0], 2))
    ax = axes[i // num_cols][i % num_cols]
    ax.scatter(x, y, label=f"{corr[0]}={corr[1]}", alpha=0.2)
    ax.set_xlabel('BG_PBE')
    ax.set_ylabel(col)
    ax.legend()

for j in range(len(numeric_cols), num_rows*num_cols):
    fig.delaxes(axes[j // num_cols][j % num_cols])

plt.tight_layout()
plt.show()

**Exploring the sine representation**

In [ ]:
#df_sine.head(3)
#display all the columns, all of the 500 features
pd.options.display.max_columns = None
df_sine.head()

In [ ]:
df_sine.info()

In [ ]:
df_sine.shape

In [ ]:
# Correlation coefficients of BG_PBE against sine numerical features.

df_sine = df_sine.set_index(['MOF'])
df_bandgaps = df_bandgaps.set_index('MOF')
df_sine_bandgap_joined = df_sine.join(df_bandgaps, how='outer', rsuffix='_bandgaps') #joining the sine fingerprints with the bandgaps data

numeric_cols = df_sine_bandgap_joined.select_dtypes(include='number').columns

df_spearman = df_sine_bandgap_joined[[col for col in numeric_cols if col.endswith(("_max", "_min")) or col == 'BG_PBE']]
df_pearson = df_sine_bandgap_joined[[col for col in numeric_cols if not col.endswith(("_max", "_min"))]]

spearman_matrix = df_spearman.corr(method='spearman')
pearson_matrix = df_pearson.corr(method='pearson')

bandgap_spearman_correlation = spearman_matrix['BG_PBE'].abs().sort_values(ascending=False)
bandgap_pearson_correlation = pearson_matrix['BG_PBE'].abs().sort_values(ascending=False)
print("Spearman:")
print(bandgap_spearman_correlation)
print("Pearson:")
print(bandgap_pearson_correlation)

In [ ]:
#These are the missing values in the sine feature representation.

df_sine_bandgap_nan = df_sine_bandgap_joined[df_sine_bandgap_joined.isna().any(axis=1)]

nan_columns = df_sine_bandgap_nan.columns[df_sine_bandgap_nan.isna().any()]
df_sine_bandgap_nan[nan_columns]

**Exploring the ofm_fp representation**

In [ ]:
df_ofm_fp.head(3)

In [ ]:
df_ofm_fp.shape

In [ ]:
df_ofm_fp

In [ ]:
# Correlation coefficients of BG_PBE against ofm_fp numerical features.

df_ofm_fp = df_ofm_fp.reset_index().set_index(['MOF'])
df_bandgaps = df_bandgaps.reset_index().set_index('MOF')
df_ofm_fp_bandgap_joined = df_ofm_fp.join(df_bandgaps, how='outer', rsuffix='_bandgaps') #joining the ofm_fp fingerprints with the bandgaps data

numeric_cols = df_ofm_fp_bandgap_joined.select_dtypes(include='number').columns

df_spearman = df_ofm_fp_bandgap_joined[[col for col in numeric_cols if col.endswith(("_max", "_min")) or col == 'BG_PBE']]
df_pearson = df_ofm_fp_bandgap_joined[[col for col in numeric_cols if not col.endswith(("_max", "_min"))]]

spearman_matrix = df_spearman.corr(method='spearman')
pearson_matrix = df_pearson.corr(method='pearson')

bandgap_spearman_correlation = spearman_matrix['BG_PBE'].abs().sort_values(ascending=False)
bandgap_pearson_correlation = pearson_matrix['BG_PBE'].abs().sort_values(ascending=False)
print("Spearman:")
print(bandgap_spearman_correlation)
print("Pearson:")
print(bandgap_pearson_correlation)

In [ ]:
#These are the missing values in the ofm_fp feature representation.

df_ofm_fp_bandgap_nan = df_ofm_fp_bandgap_joined[df_ofm_fp_bandgap_joined.isna().any(axis=1)]

nan_columns = df_ofm_fp_bandgap_nan.columns[df_ofm_fp_bandgap_nan.isna().any()]
df_ofm_fp_bandgap_nan[nan_columns]

In [ ]:
#df_qmof_refcodes.head()
df_qmof_refcodes.shape

#plotting first 10 features of 3 random MOFS against each other

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('stoich45_fingerprints.csv')

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Drop rows with NaN values
data = data.dropna()

# Randomly select three rows
random_rows = data.sample(n=3, random_state=1)  # Set random_state for reproducibility

# Extract the first 10 features (columns) from the selected rows
selected_features = random_rows.iloc[:, :10].T

# Plot the features
plt.figure(figsize=(10, 6))

# Plot each selected row with a different color
colors = ['r', 'g', 'b']
for idx, row in enumerate(selected_features.columns):
    plt.plot(selected_features.index, selected_features[row], color=colors[idx], label=f'Row {row}')

# Add labels and title
plt.xlabel('Features')
plt.ylabel('Values')
plt.title('First 10 Features for Three Random Rows')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.show()

# **Exploring 3 different dimensionality reduction techniques for the 45 and 120 fingerprints.**

**1. PCA**

45 PCA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load the dataset
data = df_45_bandgap_joined

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Ensure the data is numeric (if needed)
data = data.apply(pd.to_numeric, errors='coerce')

# Number of rows and columns before dropping NaN values
initial_rows = data.shape[0]
initial_columns = data.shape[1]

# Drop rows with NaN values
data_clean = data.dropna()

# Number of rows and columns after dropping NaN values
final_rows = data_clean.shape[0]
final_columns = data_clean.shape[1]

# Calculate and print the number of dropped rows and columns
dropped_rows = initial_rows - final_rows
dropped_columns = initial_columns - final_columns
print(f'Number of rows with NaN values dropped: {dropped_rows}')
print(f'Number of columns with NaN values dropped: {dropped_columns}')

# Splitting features and target variable
X = data_clean.drop(['BG_PBE', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'], axis=1)
y = data_clean['BG_PBE']

# Standardizing the features
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y, random_state=13, test_size=0.25, shuffle=True
)

# Applying PCA for dimensionality reduction
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)

# Print the explained variance ratio
print(f'Explained variance ratio: {pca.explained_variance_ratio_}')

# Plotting the PCA results
plt.figure(figsize=(10, 7))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='viridis', alpha=0.7)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Training Data')
plt.colorbar(label='BG_PBE')
plt.show()

pca_full = PCA().fit(scaled_data)
plt.plot(np.cumsum(pca_full.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.show()


pca 120

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load the dataset
data = df_120_bandgap_joined

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Ensure the data is numeric (if needed)
data = data.apply(pd.to_numeric, errors='coerce')

# Number of rows and columns before dropping NaN values
initial_rows = data.shape[0]
initial_columns = data.shape[1]

# Drop rows with NaN values
data_clean = data.dropna()

# Number of rows and columns after dropping NaN values
final_rows = data_clean.shape[0]
final_columns = data_clean.shape[1]

# Calculate and print the number of dropped rows and columns
dropped_rows = initial_rows - final_rows
dropped_columns = initial_columns - final_columns
print(f'Number of rows with NaN values dropped: {dropped_rows}')
print(f'Number of columns with NaN values dropped: {dropped_columns}')

# Splitting features and target variable
X = data_clean.drop(['BG_PBE', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'], axis=1)
y = data_clean['BG_PBE']

# Standardizing the features
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y, random_state=13, test_size=0.25, shuffle=True
)

# Applying PCA for dimensionality reduction
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)

# Print the explained variance ratio
print(f'Explained variance ratio: {pca.explained_variance_ratio_}')

# Plotting the PCA results
plt.figure(figsize=(10, 7))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='viridis', alpha=0.7)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Training Data')
plt.colorbar(label='BG_PBE')
plt.show()

pca_full = PCA().fit(scaled_data)
plt.plot(np.cumsum(pca_full.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.show()


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load the dataset
df_45_120_bandgap_joined_clean = df_120_bandgap_joined_clean.join(df_45_bandgap_joined_clean.drop(columns=['BG_PBE', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE']), how='right', rsuffix='_120')
data = df_45_120_bandgap_joined_clean.copy()

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Ensure the data is numeric (if needed)
data = data.apply(pd.to_numeric, errors='coerce')

# Number of rows and columns before dropping NaN values
initial_rows = data.shape[0]
initial_columns = data.shape[1]

# Drop rows with NaN values
data_clean = data.dropna()

# Number of rows and columns after dropping NaN values
final_rows = data_clean.shape[0]
final_columns = data_clean.shape[1]

# Calculate and print the number of dropped rows and columns
dropped_rows = initial_rows - final_rows
dropped_columns = initial_columns - final_columns
print(f'Number of rows with NaN values dropped: {dropped_rows}')
print(f'Number of columns with NaN values dropped: {dropped_columns}')

# Splitting features and target variable
X = data_clean.drop(['BG_PBE', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'], axis=1)
y = data_clean['BG_PBE']

# Standardizing the features
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y, random_state=13, test_size=0.25, shuffle=True
)

# Applying PCA for dimensionality reduction
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)

# Print the explained variance ratio
print(f'Explained variance ratio: {pca.explained_variance_ratio_}')

# Plotting the PCA results
plt.figure(figsize=(10, 7))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='viridis', alpha=0.7)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Training Data')
plt.colorbar(label='BG_PBE')
plt.show()

pca_full = PCA().fit(scaled_data)
plt.plot(np.cumsum(pca_full.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.show()

**2. t-SNE**

In [ ]:
# Cleaning the 45.
df_45_bandgap_nan = df_45_bandgap_joined[df_45_bandgap_joined.isna().any(axis=1)]

nan_columns = df_45_bandgap_nan.columns[df_45_bandgap_nan.isna().any()]
df_45_bandgap_joined_clean = df_45_bandgap_joined[~df_45_bandgap_joined.isna().any(axis=1)]

assert df_45_bandgap_joined_clean.shape[0] == df_45_bandgap_joined.shape[0] - df_45_bandgap_joined[df_45_bandgap_joined.isna().any(axis=1)].shape[0]

In [ ]:
# t-SNE 45
df = df_45_bandgap_joined_clean.copy()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(['BG_PBE', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'], axis=1)
y = df['BG_PBE']

scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

X_train_45, X_test_45, y_train_45, y_test_45 = train_test_split(
    X_norm, y, random_state=42, test_size=0.2, shuffle=True
)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42, perplexity=50)
X_train_45_tsne = tsne.fit_transform(X_train_45)

tsne.kl_divergence_

In [ ]:
import plotly.express as px

fig = px.scatter(x=X_train_45_tsne[:, 0], y=X_train_45_tsne[:, 1], color=y_train_45)
fig.update_layout(
    title="t-SNE visualization of 45_bandgap_joined with perplexity 50",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

In [ ]:
# Cleaning the 120
df_120_bandgap_joined_clean = df_120_bandgap_joined[~df_120_bandgap_joined.isna().any(axis=1)]

assert df_120_bandgap_joined_clean.shape[0] == df_120_bandgap_joined.shape[0] - df_120_bandgap_joined[df_120_bandgap_joined.isna().any(axis=1)].shape[0]

df_120_bandgap_joined[df_120_bandgap_joined.isna().any(axis=1)].shape[0]

In [ ]:
# t-SNE for 120
df = df_120_bandgap_joined_clean.copy()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(['BG_PBE', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'], axis=1)
y = df['BG_PBE']

scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

X_train_120, X_test_120, y_train_120, y_test_120 = train_test_split(
    X_norm, y, random_state=13, test_size=0.25, shuffle=True
)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42, perplexity=50)
X_train_120_tsne = tsne.fit_transform(X_train_120)

tsne.kl_divergence_

In [ ]:
import plotly.express as px

fig = px.scatter(x=X_train_120_tsne[:, 0], y=X_train_120_tsne[:, 1], color=y_train_120)
fig.update_layout(
    title="t-SNE visualization of 120_bandgap_joined with perplexity 50",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

In [ ]:
df_120_bandgap_joined_clean.columns

In [ ]:
df_45_bandgap_joined_clean.columns

In [ ]:
df_test1 = pd.DataFrame({
    'MOF': ['m1', 'm2', 'm3', 'm4'],
    'column1': [1,2,3,7]
})

df_test2 = pd.DataFrame({
    'MOF': ['m1', 'm2', 'm3', 'm4', 'm5'],
    'column2': [8,6,9,1,10]
})

In [ ]:
df_test1

In [ ]:
df_test2

In [ ]:
df_test2.set_index('MOF').join(df_test1.set_index('MOF'), how='left')

In [ ]:
# t-SNE for 45 and 120 combined
df_45_120_bandgap_joined_clean = df_120_bandgap_joined_clean.join(df_45_bandgap_joined_clean.drop(columns=['BG_PBE', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE']), how='right', rsuffix='_120')
df = df_45_120_bandgap_joined_clean.copy()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(['BG_PBE', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'], axis=1)
y = df['BG_PBE']

scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

X_train_45_120, X_test_45_120, y_train_45_120, y_test_45_120 = train_test_split(
    X_norm, y, random_state=13, test_size=0.25, shuffle=True
)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42, perplexity=50)
X_train_45_120_tsne = tsne.fit_transform(X_train_45_120)

tsne.kl_divergence_

In [ ]:
import plotly.express as px
# import kaleido

fig = px.scatter(x=X_train_45_120_tsne[:, 0], y=X_train_45_120_tsne[:, 1], color=y_train_45_120)
fig.update_layout(
    title="t-SNE visualization of 45_120_bandgap_joined with perplexity 50",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)

fig.show()

**3. UMAP**

In [ ]:
# Ensure the 'MOF' columns in df_45 and df_120 are set as the index for joining
df_45.set_index('MOF', inplace=True)
df_120.set_index('MOF', inplace=True)
df_bandgaps.set_index('refcode', inplace=True)

# Join the datasets with bandgaps information based on the index
joined_df_45 = df_45.join(bandgaps_df[['BG_PBE']])
joined_df_120 = df_120.join(bandgaps_df[['BG_PBE']])

# Reset the index to turn 'MOF' back into a column
joined_df_45.reset_index(inplace=True)
joined_df_120.reset_index(inplace=True)

# Check for NaN values and remove them
joined_df_45 = joined_df_45.dropna()
joined_df_120 = joined_df_120.dropna()

# Ensure 'BG_PBE' is in the columns
assert 'BG_PBE' in joined_df_45.columns, "BG_PBE not found in joined_df_45"
assert 'BG_PBE' in joined_df_120.columns, "BG_PBE not found in joined_df_120"

# Select only numeric columns for UMAP, excluding 'BG_PBE'
features_45 = joined_df_45.select_dtypes(include=[float, int]).drop(columns=['BG_PBE'])
features_120 = joined_df_120.select_dtypes(include=[float, int]).drop(columns=['BG_PBE'])

In [ ]:
# Select only numeric columns for UMAP, excluding 'BG_PBE'
umap_45 = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean').fit_transform(features_45)
umap_120 = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean').fit_transform(features_120)

In [ ]:
# Visualization function
def plot_umap(embedding, labels, title, label_name):
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=labels, cmap='Spectral', s=5)
    plt.colorbar(scatter, label=label_name)
    plt.title(title)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.show()

# Plot UMAP results with color mapping
plot_umap(umap_45, joined_df_45['BG_PBE'], 'Stoichiometric-45', 'Bandgap (eV)')
plot_umap(umap_120, joined_df_120['BG_PBE'], 'Stoichiometric-120', 'Bandgap (eV)')

In [ ]:
# Set the 'MOF' and 'refcode' columns as the index for joining
df_45.set_index('MOF', inplace=True)
df_120.set_index('MOF', inplace=True)
bandgaps_df.set_index('refcode', inplace=True)

# Join the datasets with bandgaps information using an inner join
joined_df_45 = df_45.join(bandgaps_df[['BG_PBE']], how='inner')
joined_df_120 = df_120.join(bandgaps_df[['BG_PBE']], how='inner')

# Reset the index to turn 'MOF' back into a column
joined_df_45.reset_index(inplace=True)
joined_df_120.reset_index(inplace=True)

# Rename columns to prevent conflicts
joined_df_45 = joined_df_45.rename(columns=lambda x: x + '_45' if x != 'MOF' else x)
joined_df_120 = joined_df_120.rename(columns=lambda x: x + '_120' if x != 'MOF' else x)

# Merge the two dataframes on 'MOF'
combined_df = pd.merge(joined_df_45, joined_df_120, on='MOF', how='inner')

# Check for NaN values and remove them
print("\nNaN values in combined_df before dropping:\n", combined_df.isna().sum())
combined_df = combined_df.dropna()
print("\nSize of combined_df after dropping NaN:", combined_df.shape)
print("NaN values in combined_df after dropping:\n", combined_df.isna().sum())

# Select only numeric columns for UMAP, excluding 'BG_PBE_45' and 'BG_PBE_120'
features_combined = combined_df.select_dtypes(include=[float, int]).drop(columns=['BG_PBE_45', 'BG_PBE_120'])

# Check the size of features_combined
print("\nSize of features_combined:", features_combined.shape)
print("NaN values in features_combined:\n", features_combined.isna().sum())

import umap
import matplotlib.pyplot as plt

# Perform UMAP on the combined dataset
if features_combined.shape[0] > 0:
    umap_combined = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean').fit_transform(features_combined)

    # Visualization function
    def plot_umap(embedding, labels, title, label_name):
        plt.figure(figsize=(12, 8))
        scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=labels, cmap='Spectral', s=5)
        plt.colorbar(scatter, label=label_name)
        plt.title(title)
        plt.xlabel('UMAP 1')
        plt.ylabel('UMAP 2')
        plt.show()

    # Plot combined UMAP results
    plot_umap(umap_combined, combined_df['BG_PBE_45'], 'Combined Stoichiometric-45 and 120', 'Bandgap (eV)')
else:
    print("No data available for UMAP after dropping NaN values.")

**Visuallization of comparison of 3 random MOFs against each other with 10 feautures.**

In [ ]:
df_45_bandgap_joined_clean.columns

In [ ]:
df_45_bandgap_joined_clean.index[5000:6000]

In [ ]:
df = df_45_bandgap_joined_clean.reset_index()
# Select the MOFs and features
selected_mofs = [
    'ABALOF_FSR',
    'IPEBIO01_FSR',
    'tobacco_srsb_sym_3_on_1_sym_3_mc_0_L_6'
]
selected_features = [
    'electronegativity_mean',
    'electron_affinity_mean',
    'boiling_mean',
    'ionization_energy_mean',
    'melting_mean',
    # 'Feature6',
    # 'Feature7',
    # 'Feature8',
    # 'Feature9',
    # 'Feature10'
  ]

# Filter the dataframe
filtered_df = df[df['MOF'].isin(selected_mofs)][selected_features + ['MOF']]

# Set the MOF_ID as the index for easier plotting
filtered_df.set_index('MOF', inplace=True)

# Transpose the dataframe to get features on the x-axis
filtered_df = filtered_df.T

# Plot the data
plt.figure(figsize=(12, 8))

for mof in selected_mofs:
    plt.plot(filtered_df.index, filtered_df[mof], marker='o', label=mof)

plt.xlabel('Features')
plt.ylabel('Values')
plt.title('Comparison of Selected Features for 3 MOFs')
plt.legend(title='MOFs')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


# **Training models**

 **1. Training on the 45_bandgap_clean dataframe.**

In [ ]:
# Splitting the 45_bandgap_clean df into train, validation and test
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

df = df_45_bandgap_joined_clean.reset_index().drop(columns=['MOF', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'])

X = df.drop(columns=['BG_PBE'])
y = df['BG_PBE']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

#neural network

In [ ]:
pip uninstall tensorflow
pip install tensorflow

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load and preprocess the data
data = df_45_bandgap_joined_clean.reset_index().drop(columns=['MOF', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'])

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Ensure the data is numeric
data = data.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values
data_clean = data.dropna()

# Split features and target variable
X = data_clean.drop(columns=['BG_PBE'])
y = data_clean['BG_PBE']

# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Applying PCA for dimensionality reduction (optional)
#pca = PCA(n_components=20)  # Increase the number of components for more variance retention
#X_train_pca = pca.fit_transform(X_train)
#X_valid_pca = pca.transform(X_valid)
#X_test_pca = pca.transform(X_test)

# Define the neural network model with additional layers and dropout
model = Sequential([
    Dense(256, input_dim=X_train_pca.shape[1], activation='relu'),  # Increase number of neurons
    Dropout(0.3),  # Increase dropout rate
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model with a reduced learning rate
model.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')  # Adjust learning rate

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)  # Increase patience
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=1e-6)  # Adjust patience and min_lr

# Train the model with increased epochs
history = model.fit(X_train_pca, y_train, validation_data=(X_valid_pca, y_valid), epochs=500, batch_size=32, verbose=1,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the model on the test set
loss = model.evaluate(X_test_pca, y_test)
print(f'Test loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_pca)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R² Score:", r2)

# Plot the loss over epochs
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Epochs')
plt.legend()
plt.show()

# Scatter plot of predictions vs true values
plt.figure(figsize=(10, 7))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('True BG_PBE')
plt.ylabel('Predicted BG_PBE')
plt.title('Predicted vs True BG_PBE')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Line of perfect prediction
plt.show()

# Applying PCA to the full scaled data for the cumulative explained variance plot
pca_full = PCA().fit(X_train)
plt.plot(np.cumsum(pca_full.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.show()


# without pca


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load and preprocess the data
data = df_45_bandgap_joined_clean.reset_index().drop(columns=['MOF', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'])

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Ensure the data is numeric
data = data.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values
data_clean = data.dropna()

# Split features and target variable
X = data_clean.drop(columns=['BG_PBE'])
y = data_clean['BG_PBE']

# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Define the neural network model with additional layers and dropout
model = Sequential([
    Dense(256, input_dim=X_train.shape[1], activation='relu'),  # Increase number of neurons
    Dropout(0.3),  # Increase dropout rate
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model with a reduced learning rate
model.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')  # Adjust learning rate

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)  # Increase patience
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=1e-6)  # Adjust patience and min_lr

# Train the model with increased epochs
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=500, batch_size=32, verbose=1,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')

# Make predictions
y_pred = model.predict(X_test)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R² Score:", r2)

# Plot the loss over epochs
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Epochs')
plt.legend()
plt.show()

# Scatter plot of predictions vs true values
plt.figure(figsize=(10, 7))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('True BG_PBE')
plt.ylabel('Predicted BG_PBE')
plt.title('Predicted vs True BG_PBE')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Line of perfect prediction
plt.show()
#

# automated hypertuning

In [ ]:
pip install keras-tuner


In [ ]:
import pandas as pd
import numpy as np
import kerastuner
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from kerastuner import RandomSearch

# Load and preprocess the data
data = df_45_bandgap_joined_clean.reset_index().drop(columns=['MOF', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'])

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Ensure the data is numeric
data = data.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values
data_clean = data.dropna()

# Split features and target variable
X = data_clean.drop(columns=['BG_PBE'])
y = data_clean['BG_PBE']

# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Define the neural network model with additional layers, batch normalization, and dropout
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units_1', min_value=64, max_value=512, step=32), input_dim=X_train.shape[1], activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_1', 0.2, 0.5, step=0.1)))

    model.add(Dense(hp.Int('units_2', min_value=64, max_value=512, step=32), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_2', 0.2, 0.5, step=0.1)))

    model.add(Dense(hp.Int('units_3', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_3', 0.2, 0.5, step=0.1)))

    model.add(Dense(1))

    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])),
                  loss='mean_squared_error')
    return model

# Hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=2,
    directory='my_dir',
    project_name='helloworld')

tuner.search_space_summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=1e-6)

tuner.search(X_train, y_train, epochs=500, validation_data=(X_valid, y_valid), callbacks=[early_stopping, reduce_lr])

best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the model on the test set
loss = best_model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')

# Make predictions
y_pred = best_model.predict(X_test)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R² Score:", r2)

# Plot the loss over epochs
plt.figure(figsize=(10, 7))
plt.plot(tuner.oracle.get_best_trials(num_trials=1)[0].metrics.get_history('loss'), label='Train Loss')
plt.plot(tuner.oracle.get_best_trials(num_trials=1)[0].metrics.get_history('val_loss'), label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Epochs')
plt.legend()
plt.show()

# Scatter plot of predictions vs true values
plt.figure(figsize=(10, 7))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('True BG_PBE')
plt.ylabel('Predicted BG_PBE')
plt.title('Predicted vs True BG_PBE')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Line of perfect prediction
plt.show()


#after plot  fixing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from kerastuner.tuners import RandomSearch

# Load and preprocess the data
data = df_45_bandgap_joined_clean.reset_index().drop(columns=['MOF', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'])

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Ensure the data is numeric
data = data.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values
data_clean = data.dropna()

# Split features and target variable
X = data_clean.drop(columns=['BG_PBE'])
y = data_clean['BG_PBE']

# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Define the neural network model with additional layers, batch normalization, and dropout
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units_1', min_value=64, max_value=512, step=32), input_dim=X_train.shape[1], activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_1', 0.2, 0.5, step=0.1)))

    model.add(Dense(hp.Int('units_2', min_value=64, max_value=512, step=32), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_2', 0.2, 0.5, step=0.1)))

    model.add(Dense(hp.Int('units_3', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_3', 0.2, 0.5, step=0.1)))

    model.add(Dense(1))

    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])),
                  loss='mean_squared_error')
    return model

# Hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=2,
    directory='my_dir',
    project_name='helloworld')

tuner.search_space_summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=1e-6)

tuner.search(X_train, y_train, epochs=500, validation_data=(X_valid, y_valid), callbacks=[early_stopping, reduce_lr])

best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the model on the test set
loss = best_model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')

# Make predictions
y_pred = best_model.predict(X_test)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R² Score:", r2)

# Plot the loss over epochs
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
history = best_trial.metrics.get_history()

train_loss = [metric.get('value') for metric in history['loss']]
val_loss = [metric.get('value') for metric in history['val_loss']]

plt.figure(figsize=(10, 7))
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Epochs')
plt.legend()
plt.show()

# Scatter plot of predictions vs true values
plt.figure(figsize=(10, 7))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('True BG_PBE')
plt.ylabel('Predicted BG_PBE')
plt.title('Predicted vs True BG_PBE')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Line of perfect prediction
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

data = df_45_bandgap_joined_clean.reset_index().drop(columns=['MOF', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'])

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Ensure the data is numeric
data = data.apply(pd.to_numeric, errors='coerce')

# Number of rows and columns before dropping NaN values
initial_rows = data.shape[0]
initial_columns = data.shape[1]

# Drop rows with NaN values
data_clean = data.dropna()

# Number of rows and columns after dropping NaN values
final_rows = data_clean.shape[0]
final_columns = data_clean.shape[1]

# Calculate and print the number of dropped rows and columns
dropped_rows = initial_rows - final_rows
dropped_columns = initial_columns - final_columns
print(f'Number of rows with NaN values dropped: {dropped_rows}')
print(f'Number of columns with NaN values dropped: {dropped_columns}')

# Check if essential columns exist after cleaning
#essential_columns = ['BG_PBE', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE']
#missing_columns = [col for col in essential_columns if col not in data_clean.columns]
#if missing_columns:
#    raise ValueError(f"The following essential columns are missing after cleaning: {missing_columns}")

# Splitting features and target variable
#X = data_clean.drop(essential_columns, axis=1)
#y = data_clean['BG_PBE']

# Standardizing the features
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

# Splitting data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(
#    X_norm, y, random_state=13, test_size=0.25, shuffle=True
#)

# Applying PCA for dimensionality reduction (optional)
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_pca.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Output layer for regression

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
history = model.fit(X_train_pca, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test_pca, y_test)
print(f'Test loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_pca)

# Plotting the PCA results
plt.figure(figsize=(10, 7))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='viridis', alpha=0.7)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Training Data')
plt.colorbar(label='BG_PBE')
plt.show()

# Applying PCA to the full scaled data for the cumulative explained variance plot
pca_full = PCA().fit(X_norm)
plt.plot(np.cumsum(pca_full.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.show()



mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R² Score:", r2)

# Plot the loss over epochs
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Epochs')
plt.legend()
plt.show()



#mse = mean_squared_error(y_valid, y_pred)
#r2 = r2_score(y_valid, y_pred)

#print(f'Mean Squared Error: {mse}')
#print(f'R^2 Score: {r2}')
#from sklearn.metrics import precision_score
#precision = precision_score(y_valid, y_pred)
#print(f"Precision: {precision:.2f}")
#from sklearn.metrics import accuracy_score
#accuracy = accuracy_score(y_valid, y_pred)
#print(f"Accuracy: {accuracy:.2f}")

# Scatter plot of predictions vs true values
plt.figure(figsize=(10, 7))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('True BG_PBE')
plt.ylabel('Predicted BG_PBE')
plt.title('Predicted vs True BG_PBE')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)  # Line of perfect prediction
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load and preprocess the data
data = df_45_bandgap_joined_clean.reset_index().drop(columns=['MOF', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'])

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Ensure the data is numeric
data = data.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values
data_clean = data.dropna()

# Split features and target variable
X = data_clean.drop(columns=['BG_PBE'])
y = data_clean['BG_PBE']

# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Applying PCA for dimensionality reduction (optional)
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train)
X_valid_pca = pca.transform(X_valid)
X_test_pca = pca.transform(X_test)

# Define the neural network model
model = Sequential([
    Dense(128, input_dim=X_train_pca.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

# Train the model
history = model.fit(X_train_pca, y_train, validation_data=(X_valid_pca, y_valid), epochs=200, batch_size=32, verbose=1,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the model on the test set
loss = model.evaluate(X_test_pca, y_test)
print(f'Test loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_pca)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R² Score:", r2)

# Plot the loss over epochs
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Epochs')
plt.legend()
plt.show()

# Scatter plot of predictions vs true values
plt.figure(figsize=(10, 7))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('True BG_PBE')
plt.ylabel('Predicted BG_PBE')
plt.title('Predicted vs True BG_PBE')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Line of perfect prediction
plt.show()

# Applying PCA to the full scaled data for the cumulative explained variance plot
pca_full = PCA().fit(X_norm)
plt.plot(np.cumsum(pca_full.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.show()


Convolutional neural network

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf




data = df_45_bandgap_joined_clean.reset_index().drop(columns=['MOF', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'])

# Exclude the first row and first column
data = data.iloc[1:, 1:]


# Drop rows with NaN values
data_clean = data.dropna()

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Reshape the data for CNN input
X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_valid = X_valid.reshape(-1, X_valid.shape[1], 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)

# Create a neural network model for regression
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100, batch_size=32)

# Evaluate the model on the test set
y_pred = model.predict(X_test)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R² Score:", r2)

# Plot the loss over epochs
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Epochs')
plt.legend()
plt.show()

# Scatter plot of predictions vs true values
plt.figure(figsize=(10, 7))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('True BG_PBE')
plt.ylabel('Predicted BG_PBE')
plt.title('Predicted vs True BG_PBE')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Line of perfect prediction
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load and preprocess the data
data = df_45_bandgap_joined_clean.reset_index().drop(columns=['MOF', 'CBM_PBE', 'VBM_PBE', 'Direct_PBE'])

# Exclude the first row and first column
data = data.iloc[1:, 1:]

# Drop rows with NaN values
data_clean = data.dropna()

# Separate features and target variable
X = data_clean.drop(columns=['BG_PBE'])
y = data_clean['BG_PBE']

# Split the data into train, validation, and test sets
#X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

X = X.sample(frac=1, axis=1, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Reshape the data for CNN input
X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_valid = X_valid.reshape(-1, X_valid.shape[1], 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)

# Create a neural network model for regression
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Conv1D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Define a learning rate scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

# Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=200, batch_size=64, callbacks=[lr_scheduler, early_stopping], verbose=1)

# Evaluate the model on the test set
y_pred = model.predict(X_test)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R² Score:", r2)

# Plot the loss over epochs
plt.figure(figsize=(10, 7))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Epochs')
plt.legend()
plt.show()

# Scatter plot of predictions vs true values
plt.figure(figsize=(10, 7))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('True BG_PBE')
plt.ylabel('Predicted BG_PBE')
plt.title('Predicted vs True BG_PBE')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Line of perfect prediction
plt.show()

**Selection of Models** : Random Forest, Gaussian Process Regressor, XGB Regressor, Multilayer perceptron NN, Ridge Regression, Lasso.

In [ ]:
'''
Shortlist the best performing models. We aim to choose 3 models to proceed further.
'''
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
                    ("scaler", StandardScaler()),
                    ("regressor", None)
                    ])

param_grid = {'regressor': [
                             RandomForestRegressor(n_estimators=100),
                             GaussianProcessRegressor(),
                             Ridge(),
                             Lasso(),
                             XGBRegressor(),
                             MLPRegressor()
                             ]
              }

grid_search = GridSearchCV(pipeline,
                           param_grid,
                           cv=5,
                           scoring='r2',
                           return_train_score=True
                           )
grid_search.fit(X_train_val, y_train_val)

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.cv_results_

 'mean_test_score': array([ 6.94196553e-01, -3.36830726e+03,  5.12576164e-01, -1.84752611e-04,
         6.84328776e-01,  6.55321024e-01]),

Performance of models on the 45_bandgap:

1.   Random Forest: 0,69
2.   XGB Regressor: 0,68
3.   MLP Regressor: 0,65
4.   Ridge: 0,51
5.   Lasso: -0,0001
6.   Gaussian Process Regressor: -3000






Training on Dimension reduction

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

pca = PCA(n_components=10)

pipe_pca = Pipeline([('PCA', pca),
                 ('regressor', XGBRegressor())])

cv_xgb_pca = cross_validate(pipe_pca, X_train_val, y_train_val, cv=5, n_jobs=-1)
cv_xgb_pca['test_score']

In [ ]:
#XGB Regressor with PCA 45 Hyperparameter optimisation

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.decomposition import PCA

param_grid = {
    #"regressor__n_estimators":[100, 300, 500] ,
    #"regressor__max_depth": [6, 8, 10],
    #"regressor__min_samples_split": 5,
    #"regressor__learning_rate": [0.1, 0.05, 0.01],
    #"regressor__loss": "squared_error",
}
pca = PCA(n_components=10)

reg = Pipeline([('PCA', pca),
                    ("scaler", StandardScaler()),
                    ("regressor", XGBRegressor(n_estimators=500, max_depth=8, learning_rate=0.05)) #GradientBoostingRegressor())
                    ])

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1, return_train_score=True)

# Fit GridSearchCV to the training data
grid_search.fit(X_train_val, y_train_val)

# Retrieve the best parameters
best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')


**Perfomance of the XGB Regressor with PCA on the 45df with optimal hyperparameters on the validation set is: 0.62145388**

**Optimal hyperparameters are: n_estimators=500, max_depth=8, learning_rate=0.05**

Evalutaion metric for hyperparameter optimisation used is r^2.

In [ ]:
# Do this at the end for XGB
reg = Pipeline([('PCA', pca),
                    ("scaler", StandardScaler()),
                    ("regressor", XGBRegressor(n_estimators=300, max_depth=8, learning_rate=0.05)) # insert your best parameters here
                    ])

reg.fit(X_train_val, y_train_val)

y_pred = reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

In [ ]:
x = y_test
y = y_pred

plt.scatter(x, y, alpha=0.4, label=f"r^2 = {round(r2, 2)}, RMSE = {round(np.sqrt(mse), 2)}")
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('XGB with PCA on the test set for 45_bandgap')
plt.legend()
plt.show()

1. **Random Forest**

In [ ]:
assert X_train.shape[0] + X_valid.shape[0] + X_test.shape[0] == df.shape[0]

In [ ]:
rf = RandomForestRegressor(n_estimators=100)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_valid)

mse = mean_squared_error(y_valid, y_pred)
r2 = r2_score(y_valid, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

In [ ]:
plt.scatter(y_valid, y_pred)

plt.ylabel('Predicted')
plt.xlabel('Measured')
plt.show()

In [ ]:
# Random Forest 45 Hyperparameter optimisation

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define the parameter grid for GridSearchCV
param_grid = {
    #'regressor__n_estimators': [1000],
    #'regressor__max_features': [1.0, 'sqrt', 'log2']
    'regressor__max_depth': [8, 10, 12],
    'regressor__min_samples_split': [4, 6, 8],
    'regressor__min_samples_leaf': [3, 5, 7]
}

rf = Pipeline([
                    ("scaler", StandardScaler()),
                    ("regressor", RandomForestRegressor(n_estimators=1000, max_features=1.0))
                    ])

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1,  return_train_score=True)

# Fit GridSearchCV to the training data
grid_search.fit(X_train_val, y_train_val)

# Retrieve the best parameters
best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')

In [ ]:
# Results for all parameter combinations, lookibg fo the best performance
grid_search.cv_results_

Extra trees on 45

In [ ]:
# Extra Trees 45 Hyperparameter optimisation
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Define the parameter grid for GridSearchCV
param_grid = {
    #'regressor__n_estimators': [1000],
    #'regressor__max_features': [1.0, 'sqrt', 'log2']
    #'regressor__max_depth': [18, 23, 28],
    #'regressor__min_samples_split': [4, 6],
    #'regressor__min_samples_leaf': [3, 5]
}

rf = Pipeline([
                    ("scaler", StandardScaler()),
                    ("regressor", ExtraTreesRegressor(n_estimators=1000, max_features=1.0, min_samples_leaf=3, min_samples_split=4, max_depth=18))
                    ])

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1,  return_train_score=True)

# Fit GridSearchCV to the training data
grid_search.fit(X_train_val, y_train_val)

# Retrieve the best parameters
best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')

In [ ]:
grid_search.cv_results_

**Perfomance of the Extra Trees Regressor on the 45df with optimal hyperparameters on the validation set is mean_test_score: 0.699859**

**Optimal hyperparameters are: n_estimators=1000, max_features=1.0, min_samples_leaf=3, min_samples_split=4, max_depth=18**

In [ ]:
# Do this at the end for ET
rf = Pipeline([ ("scaler", StandardScaler()),
                    ("regressor", ExtraTreesRegressor(n_estimators=1000, max_features=1.0, min_samples_leaf=3, min_samples_split=4, max_depth=18)) # insert your best parameters here
                    ])

rf.fit(X_train_val, y_train_val)

y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

In [ ]:
x = y_test
y = y_pred

plt.scatter(x, y, alpha=0.4, label=f"r^2 = {round(r2, 2)}, RMSE = {round(np.sqrt(mse), 2)}")
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Extra Trees on the test set for 45_bandgap')
plt.legend()
plt.show()

Extra Trees with PCA on 45

In [ ]:
#Extra Trees with PCA 45 Hyperparameter optimisation

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.decomposition import PCA

param_grid = {
     'regressor__n_estimators': [300, 500, 1000],
    'regressor__max_features': [1.0, 'sqrt', 'log2'],
    'regressor__max_depth': [16, 18, 23],
    'regressor__min_samples_split': [2, 4, 6],
    'regressor__min_samples_leaf': [3, 5]
}
pca = PCA(n_components=10)

et = Pipeline([("scaler", StandardScaler()),
               ('PCA', pca),
               ("regressor", ExtraTreesRegressor())
            ])

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=et, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1, return_train_score=True)

# Fit GridSearchCV to the training data
grid_search.fit(X_train_val, y_train_val)

# Retrieve the best parameters
best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')

2. **Gaussian process regression**


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV

param_grid = {
    #"regressor__kernel": [RBF(), None],
    # "regressor__alpha": [1e-5, 1e-4, 1e-3, 0.01, 0.1],
    "regressor__alpha": [0.1, 1, 10],
    #"regressor__optimizer": ["fmin_l_bfgs_b", "fmin_ncg"],
    #"regressor__n_restarts_optimizer": [0, 1, 2]
}

# Initialize GaussianProcessRegressor
gpr = Pipeline([
                    ("scaler", StandardScaler()),
                    ("regressor", GaussianProcessRegressor())
                    ])

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=gpr, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit GridSearchCV to the training data
grid_search.fit(X_train_val, y_train_val)

# Retrieve the best parameters
best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')

In [ ]:
# Results for all parameter combinations, lookibg fo the best performance
grid_search.cv_results_

In [ ]:
# Do this at the end for Gaussian
gpr = Pipeline([
                    ("scaler", StandardScaler()),
                    ("regressor", GaussianProcessRegressor(kernel=None, alpha=1e-10, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=0)) # insert your best parameters here
                    ])

gpr.fit(X_train_val, y_train_val)

y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

3. **XGB Regressor**

In [ ]:
#XGB Regressor 45 Hyperparameter optimisation

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
#from sklearn.ensemble import GradientBoostingRegressor

param_grid = {
    "regressor__n_estimators":[100, 300, 500] ,
    "regressor__max_depth": [2, 4, 6],
    #"regressor__min_samples_split": 5,
    "regressor__learning_rate": [0.1, 0.01, 0.001, 0.0001],
    #"regressor__loss": "squared_error",
}


reg = Pipeline([
                    ("scaler", StandardScaler()),
                    ("regressor", XGBRegressor()) #GradientBoostingRegressor())
                    ])

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit GridSearchCV to the training data
grid_search.fit(X_train_val, y_train_val)

# Retrieve the best parameters
best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')


First time run results: Best parameters found: {'regressor__learning_rate': 0.1, 'regressor__max_depth': 6, 'regressor__n_estimators': 300}

In [ ]:
# Results for all parameter combinations, lookibg fo the best performance
grid_search.cv_results_

In [ ]:

param_grid = {
    #"regressor__n_estimators":[100, 300, 500] ,
    #"regressor__max_depth": [6, 8, 10],
    #"regressor__min_samples_split": 5,
    "regressor__learning_rate": [0.5, 0.1, 0.05, 0.01],
    #"regressor__loss": "squared_error",
}


reg = Pipeline([
                    ("scaler", StandardScaler()),
                    ("regressor", XGBRegressor(n_estimators=300, max_depth=8)) #GradientBoostingRegressor())
                    ])

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit GridSearchCV to the training data
grid_search.fit(X_train_val, y_train_val)

# Retrieve the best parameters
best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')

In [ ]:
# Results for all parameter combinations, lookibg fo the best performance
grid_search.cv_results_

**Perfomance of the XGB Regressor on the 45df with optimal hyperparameters on the validation set is: 0.70288743**

**Optimal hyperparameters are: n_estimators=300, max_depth=8, learning_rate=0.05**

In [ ]:
# Do this at the end for XGB
gpr = Pipeline([
                    ("scaler", StandardScaler()),
                    ("regressor", XGBRegressor(n_estimators=300, max_depth=8, learning_rate=0.05)) # insert your best parameters here
                    ])

gpr.fit(X_train_val, y_train_val)

y_pred = gpr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

In [ ]:
x = y_test
y = y_pred

plt.scatter(x, y, alpha=0.4, label=f"r^2 = {round(r2, 2)}, RMSE = {round(np.sqrt(mse), 2)}")
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('XGBoost on the test set')
plt.legend()
plt.show()

4. **MLP Regressor**

In [ ]:
#We dont need this right now, only later when modelling.
#Joining data sets into one data frame

df_120 = df_120.set_index(['MOF'])
df_sine = df_sine.set_index(['MOF'])
#df_qmof_refcodes = df_qmof_refcodes.set_index(['MOF'])
df_ofm_fp = df_ofm_fp.set_index(['MOF'])


df_joinn = df_45.join(df_120, how='outer', rsuffix='_120').join(df_sine, how='outer', rsuffix='_sine').join(df_ofm_fp, how='outer', rsuffix='ofm_fp')
